# Brugia Viral Search

The goal of this analysis is to identify and hopefully assemble to some degree the viruses that are found present in the Brugia RNA-Seq data.

The first step is to run kraken on the data to identify likely viruses from the RefSeq database. Given it is limited to the RefSeq database, it is surely not comprehensive enough to be extrememly beneficial in de novo assembly, so LMAT is going to be run as well.

The Brugia data have been filtered out via tophat which may or may not be suffice. We will have to see

In [ ]:
#!/bin/bash
#PBS -l walltime=24:00:00,nodes=1:ppn=20,mem=96gb
#PBS -j oe
#PBS -m a
#PBS -M twaddlac@gmail.com

module load kraken
cd /scratch/at120/brugia-rnaseq/viral-search

kraken --db /scratch/at120/db/kraken/standard --threads 20 --fastq-input --output all-unmapped.kraken.out --preload all-unmapped.r1.fastq all-unmapped.r2.fastq
kraken-translate --db /scratch/at120/db/kraken/standard > kraken.translate.out
kraken-mpa-report --db /scratch/at120/db/kraken/standard > kraken.mpa.out

Once kraken has generated output, obtaining the reads that are viral and not bactieria, arcahea, or phiX can be done with the following command

In [ ]:
grep 'Virus' kraken.translate.out | grep -v 'phiX' | cut -f1 > viral-read-names.txt